In [70]:
import scanpy as sc
import numpy as np
import numba
import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter, defaultdict
import itertools as it
from utils import *

In [71]:
FILE = "datasets/10XGenomics/Targeted_SC3v3_Human_Glioblastoma_Neuroscience_filtered_feature_bc_matrix.h5"

In [72]:
adata = sc.read_10x_h5(FILE)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [73]:
def reclassify(X, N, override=[]):
    X = np.array(X)
    data = np.array(X[X >= 1]).flatten()
    c = Counter(data)
    chunk = len(data) // N
    vals = [[] for _ in range(N)]
    i = 0
    curr = 0
    for k in sorted(c):
        vals[i].append(k)
        curr += c[k]
        if curr >= chunk:
            i += 1
            curr = 0

    vals = [[i[0], i[-1]] for i in vals if i]
    last = vals.pop(-1)
    diff = int(N - len(vals))
    left, right = last[0], last[-1]
    size = 10
    if diff == 0 or diff == 1:
        vals.append([left, right])
    else:
        B = (right - left - diff * size) // ((diff * (diff - 1)) / 2)
        print(left, right)
        for i in range(diff):
            vals.append([left, left + size])
            left += size + 1
            size += B
        if left < right:
            vals.append([left + 1, right])
    # for k in range(left, right, (right - 
    
    # vals = [[i[0], i[-1]] for i in vals if i]
    print(vals)
    # body = 1
    # while body in c:
    #     body += 1
    
    # chunk = (body + 1) // N + 1
    # vals = [[] for _ in range(N)]
    # i = 0
    # curr = 0
    # for k in range(1, body + 1):
    #     vals[i].append(k)
    #     if len(vals[i]) >= chunk:
    #         i += 1
    # vals.append([body + 1, max(c) + 1])
    if override:
        vals = override
    # print(vals)
    for val, chunk_boundaries in enumerate(vals): # tqdm.tqdm(enumerate(vals)):
        if len(chunk_boundaries) == 0:
            continue

        l, r = chunk_boundaries[0], chunk_boundaries[-1]
        # v = 1 if val == 0 else r # (l + r) // 2
        v = val + 1
        # print("chunking", l, r, val + 1)
        X = np.where((l <= X) & (X <= r), v, X)
    return X
# reclassify(sc.read_10x_h5(FILE).X.todense(), 20)

In [74]:
Q = adata.X.todense()

In [75]:
adata = ad.AnnData(reclassify(Q, 3))
X = adata.X
np.count_nonzero(X)

[[1.0, 1.0], [2.0, 4.0], [5.0, 2551.0]]


1850701

In [76]:
X

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 3., ..., 1., 0., 0.],
       ...,
       [0., 0., 2., ..., 0., 0., 0.],
       [1., 0., 3., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [77]:
Q

matrix([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  0., 13., ...,  1.,  0.,  0.],
        ...,
        [ 0.,  0.,  2., ...,  0.,  0.,  0.],
        [ 1.,  0., 22., ...,  1.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [20]:
def cell(i):
    return np.array(X[i, :]).flatten()
ncells = X.shape[0]
sims = np.zeros((ncells, ncells))

cells = []
for i in tqdm.trange(ncells):
    cells.append(cell(i))


100%|███████████████████████████████████| 5697/5697 [00:00<00:00, 468997.43it/s]


In [124]:
@numba.jit(nopython=True, parallel=True)
def compute_similarities_numba(cells):
    ncells = cells.shape[0]
    sims = np.zeros((ncells, ncells))
    
    for i in numba.prange(ncells):
        a = cells[i]
        q = a != 0
        
        for j in range(i+1, ncells): 
            b = cells[j]
            similarity = np.sum(q & (a == b))
            # similarity = np.count_nonzero(a - b)
            sims[i, j] = similarity
            sims[j, i] = similarity  
    
    return sims

In [79]:
compute_similarities_numba(X)

array([[  0.,   7.,  11., ...,  12.,  14.,  15.],
       [  7.,   0.,  13., ...,  25.,  13.,  20.],
       [ 11.,  13.,   0., ...,  46., 188.,  69.],
       ...,
       [ 12.,  25.,  46., ...,   0.,  49.,  75.],
       [ 14.,  13., 188., ...,  49.,   0.,  62.],
       [ 15.,  20.,  69., ...,  75.,  62.,   0.]])

In [47]:
for i in tqdm.trange(ncells):
    a = cells[i]
    q = a != 0
    for j in range(ncells):
        if i == j:
            sims[i][j] = 0
        elif sims[j][i] != 0:
            sims[i][j] = sims[j][i]
        else:
            b = cells[j]
            sims[i][j] = sum(q & (a == b))

100%|██████████████████████████████████████| 5697/5697 [00:09<00:00, 604.83it/s]


In [48]:
sims

array([[  0.,   7.,  11., ...,  12.,  14.,  15.],
       [  7.,   0.,  13., ...,  25.,  13.,  20.],
       [ 11.,  13.,   0., ...,  46., 188.,  69.],
       ...,
       [ 12.,  25.,  46., ...,   0.,  49.,  75.],
       [ 14.,  13., 188., ...,  49.,   0.,  62.],
       [ 15.,  20.,  69., ...,  75.,  62.,   0.]])

In [ ]:
with open('sims.npy', 'wb') as f:
    np.save(f, sims)

In [39]:
i = 2
x = np.quantile(list(sims[i][:i]) + list(sims[i][i + 1:]), 0.9)
L = cells[i].shape[0]
out = np.full(L, True)
out &= cells[i] != 0

s_arr = list(zip(range(L), sims[i]))

q = 0
for k, v in sorted(s_arr, key = lambda i: -i[1]):
    if k == i or v < x:
        continue

    # print(k, v)
    out &= cells[i] == cells[k]
    print(sum(out))
    q += 1
print(q)

412
396
375
359
343
332
323
311
304
298
293
287
282
278
271
269
264
260
253
247
244
236
234
232
229
226
220
217
213
211
210
206
205
201
200
199
198
198
195
194
191
189
188
186
184
181
180
177
177
175
173
170
167
166
164
162
160
160
159
158
156
154
154
153
153
153
151
150
149
148
147
147
145
145
143
141
140
140
140
139
138
138
138
136
136
136
135
134
134
133
133
133
133
131
130
130
129
127
127
127
127
124
122
121
121
121
121
121
120
119
119
119
118
118
118
118
117
117
117
116
114
113
112
109
109
109
108
106
106
105
105
104
102
133


In [29]:
# Pick a number R representatives
# Everybody picks their best friend
# Take the top R most picked as their representatives

best_friends = [np.argmax(row) for row in sims]
# for k, row in enumerate(sims):
#     # other_than = list(sims[k][:k]) + list(sims[k][k + 1:])
#     best_friends.append(row.index(max(row)))
counter = Counter(best_friends)
counter

Counter({3863: 34,
         4189: 29,
         1247: 28,
         260: 26,
         5093: 25,
         3901: 25,
         2630: 22,
         2664: 22,
         4811: 21,
         4696: 21,
         5576: 20,
         4465: 20,
         269: 20,
         4731: 20,
         4832: 19,
         544: 19,
         1901: 19,
         332: 18,
         4000: 18,
         3356: 18,
         4248: 17,
         3542: 17,
         4423: 17,
         140: 17,
         3771: 16,
         2996: 16,
         4030: 16,
         1601: 16,
         4809: 16,
         2234: 15,
         4511: 15,
         4057: 15,
         656: 15,
         5009: 15,
         1823: 15,
         4399: 15,
         1377: 15,
         70: 14,
         5540: 14,
         3960: 14,
         1328: 14,
         3121: 14,
         127: 14,
         1107: 14,
         290: 14,
         854: 14,
         3774: 13,
         368: 13,
         5527: 13,
         755: 13,
         2054: 13,
         4992: 13,
         2032: 13,
      

In [28]:
sims[0][264]

27.0

In [36]:
choices = [-1 for _ in range(len(cells))]
R = 31
representatives = [i[0] for i in counter.most_common(R)]

for k, friend in enumerate(best_friends):
    if k in representatives:
        continue # representatives have no friends
        
    if friend in representatives:
        choices[k] = friend
    else:
        best_score, best_friend = None, None
        for r in representatives:
            if best_score is None or sims[k][r] > best_score:
                best_score = sims[k][r]
                best_friend = r

        choices[k] = best_friend

In [136]:
choices

[3542,
 2664,
 4465,
 544,
 4511,
 332,
 3901,
 4000,
 1601,
 4465,
 4731,
 3356,
 140,
 5093,
 4248,
 332,
 269,
 4696,
 260,
 4189,
 140,
 332,
 2664,
 4465,
 4696,
 4248,
 2234,
 3356,
 1601,
 3901,
 1901,
 332,
 332,
 3356,
 4811,
 332,
 4030,
 2630,
 269,
 2996,
 5093,
 140,
 4030,
 4511,
 269,
 2234,
 4000,
 3542,
 1247,
 4731,
 5093,
 5093,
 4465,
 332,
 1901,
 3863,
 4511,
 4248,
 5576,
 4465,
 2664,
 2630,
 1247,
 4511,
 4731,
 3901,
 5576,
 544,
 5093,
 1247,
 4511,
 3901,
 140,
 2234,
 4809,
 3771,
 3863,
 332,
 1247,
 4511,
 544,
 4511,
 3542,
 269,
 2664,
 1601,
 3542,
 4000,
 1901,
 4030,
 4511,
 269,
 544,
 4811,
 3901,
 5093,
 4511,
 544,
 5576,
 4423,
 2630,
 4731,
 140,
 3771,
 269,
 3863,
 332,
 2996,
 4832,
 4811,
 5093,
 3542,
 260,
 1247,
 4731,
 4030,
 3356,
 140,
 1901,
 544,
 3542,
 140,
 3901,
 2664,
 544,
 332,
 3356,
 3863,
 140,
 1901,
 3356,
 4511,
 2630,
 140,
 4809,
 332,
 4511,
 3863,
 2664,
 544,
 -1,
 3863,
 4511,
 4731,
 332,
 2234,
 5576,
 332,
 124

In [40]:
new_cells = np.zeros(X.shape)
for k, choice in enumerate(choices):
    new_cells[k] = (cells[choice] - cells[k]) % 3  

In [128]:
new_cells

array([[0., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 2., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [120]:
np.count_nonzero(new_cells)

1645562

In [157]:
def diffcompress(Q, R, B):
    adata = ad.AnnData(reclassify(Q, B))
    X = adata.X
    sims = compute_similarities_numba(X)
    print(sims)

    # Pick a number R representatives
    # Everybody picks their best friend
    # Take the top R most picked as their representatives
    
    best_friends = [np.argmax(row) for row in sims]
    counter = Counter(best_friends)

    choices = [-1 for _ in range(len(X))]
    representatives = [i[0] for i in counter.most_common(R)]

    if not representatives:
        return X
        
    for k, friend in enumerate(best_friends):
        if k in representatives:
            continue # representatives have no friends
            
        if friend in representatives:
            choices[k] = friend
        else:
            best_score, best_friend = None, None
            for r in representatives:
                if best_score is None or sims[k][r] > best_score:
                    best_score = sims[k][r]
                    best_friend = r
    
            choices[k] = best_friend
    # print(choices)
    new_cells = np.zeros(X.shape)
    for k, choice in enumerate(choices):
        new_cells[k] = X[k]
        if choice == -1:
            new_cells[k] = X[k]
        else:
            score = np.count_nonzero(X[choice] - X[k]) - np.count_nonzero(X[k])
            if score < 0:
                new_cells[k] = (X[choice] - X[k]) % B
            else:
                new_cells[k] = X[k]

    return new_cells

In [149]:
np.count_nonzero(Q)

1850701

In [160]:
np.count_nonzero(diffcompress(Q, 4, 2))

[[1.0, 2.0], [3.0, 2551.0]]
[[  0.  11.  22. ...  16.  23.  25.]
 [ 11.   0.  26. ...  36.  21.  38.]
 [ 22.  26.   0. ...  70. 225.  97.]
 ...
 [ 16.  36.  70. ...   0.  83.  99.]
 [ 23.  21. 225. ...  83.   0. 103.]
 [ 25.  38.  97. ...  99. 103.   0.]]


1561732

In [156]:
1517312 / 1850701

0.8198579889458103